In [25]:
import pandas as pd
import math
import ast
import numpy as np
import re
from sklearn.model_selection import train_test_split

Store the data in two lists that contains the different lines

In [26]:
with open('C:/Users/juan_/Desktop/programación/NLP_ML_DL_UdemyCourse/documents_for_course/Edgar_allan_poe.txt',encoding="utf-8", mode = "r") as doc:
    poe_dataset_noisy = doc.readlines()

In [27]:
with open('C:/Users/juan_/Desktop/programación/NLP_ML_DL_UdemyCourse/documents_for_course/Robert_frost.txt',encoding="utf-8", mode = "r") as doc:
    frost_dataset_noisy = doc.readlines()

We're gonna preprocess the text so it doesn't take 'The' or 'the' as different states cause they are actually the same. Also we will take a look at punctuation and remove it as well as the empty lines

In [28]:
def clean_dataset(dataset):
    clean_dataset = []
    for line in dataset:
        line = line.lower()
        line = re.sub(r"([-.,\\/#!$%\\^&\\*;:{}=\\-_`~()])","",line)
        line = re.sub(r"\n","",line)
        line = line.strip()
        clean_dataset.append(line)
    for line in clean_dataset:
        if line == '':
            clean_dataset.remove(line)
    return clean_dataset

In [29]:
poe_dataset = clean_dataset(poe_dataset_noisy)
frost_dataset = clean_dataset(frost_dataset_noisy)
print(len(poe_dataset))
print(len(frost_dataset))
print(frost_dataset[100:110])

718
1436
['in the confusion of them all aglitter', 'and birds that joined in the excited fun', 'by doubling and redoubling song and twitter', "i have no doubt i'd end by holding none", "it takes the moon for this the sun's a wizard", "by all i tell but so's the moon a witch", 'from the high west she makes a gentle cast', 'and suddenly without a jerk or twitch', 'she has her speel on every single lizard', "i fancied when i looked at six o'clock"]


In [30]:
unique_dataset = []
labels = []

def create_unique_dataset_poe(dataset):
    for line in dataset:
        unique_dataset.append(line)
        labels.append(0)

def create_unique_dataset_frost(dataset):
    for line in dataset:
        unique_dataset.append(line)
        labels.append(1)

create_unique_dataset_poe(poe_dataset)
create_unique_dataset_frost(frost_dataset)

In [31]:
len(unique_dataset)

2154

We need to create a dataset that contains both of the authors because they are completly imbalanced as we can see from the cell on the top

Create a function that generates a dataframe for each author and split that dataframe into testing and training 

In [32]:
def divide_set(dataset):
    dataframe_set = pd.DataFrame({'Author': pd.Series(dtype='str'),'Poem': pd.Series(dtype='str')})
    for sentence in range(len(dataset)):
        list_row = [labels[sentence],dataset[sentence]]
        dataframe_set.loc[len(dataframe_set)] = list_row
        

    training_data = dataframe_set.sample(frac=0.8, random_state=25)
    testing_data = dataframe_set.drop(training_data.index)

    return training_data, testing_data


In [33]:
# divide_set(unique_dataset,labels)

TypeError: divide_set() takes 1 positional argument but 2 were given

Generate with our function out training and testing datasets

In [ ]:
poe_training_data, poe_testing_data = divide_set(poe_dataset)
frost_training_data, frost_testing_data = divide_set(frost_dataset)
# print(poe_training_data.head())

TypeError: divide_set() missing 1 required positional argument: 'labels'

Create a function that, given a specific dataframe, returns a word to index mapping for every word in the dataset

In [ ]:
def get_vocabulary_size(dataset):
    number_of_states = {}
    counter = 0
    for index, row in dataset.iterrows():
        poem_line = row['Poem']
        for word in poem_line.split(" "):
            if word not in number_of_states:
                number_of_states[word] = counter
                counter +=1
    number_of_states["unknown_word"] = len(number_of_states) + 1
    return number_of_states

In [ ]:
states_in_poe = get_vocabulary_size(poe_training_data)
states_in_frost = get_vocabulary_size(frost_training_data)
print(states_in_poe)
# print([i for i in states_in_poe if i == 14])

{'said': 0, "'sadly": 1, 'this': 2, 'star': 3, 'i': 4, 'mistrust': 5, "'tis": 6, 'as': 7, 'plain': 8, 'the': 9, 'light': 10, 'of': 11, 'day': 12, 'that': 13, 'you': 14, 'double': 15, 'it': 16, 'and': 17, 'tempted': 18, 'her': 19, 'out': 20, 'gloom': 21, 'which': 22, 'doth': 23, 'now': 24, 'know': 25, 'she': 26, 'has': 27, 'seen': 28, 'tears': 29, 'are': 30, 'not': 31, 'dry': 32, 'on': 33, 'drear': 34, 'path': 35, 'alas': 36, 'where': 37, 'grows': 38, 'those': 39, 'unusual': 40, 'strings': 41, 'save': 42, 'only': 43, 'thee': 44, 'me': 45, 'paused': 46, 'looked': 47, 'distinct': 48, 'with': 49, 'its': 50, 'duplicate': 51, 'horn': 52, 'purified': 53, 'in': 54, 'their': 55, 'electric': 56, 'fire': 57, 'deep': 58, 'thoughts': 59, 'a': 60, 'duty': 61, 'winged': 62, 'odour': 63, 'went': 64, 'away': 65, 'to': 66, 'discordant': 67, 'melody': 68, 'see': 69, 'them': 70, 'still': 71, 'two': 72, 'sweetly': 73, 'scintillant': 74, 'who': 75, 'alterest': 76, 'all': 77, 'things': 78, 'thy': 79, 'peerin

Now we have to turn every line of text in the training dataset into its respective series of integers from our mapping. 
Then we create a markov model for each of the classes

In [ ]:
def transform_dataset_to_index_mapping(dataset, states_map):
    dataframe_indexed_set = pd.DataFrame({'Line': pd.Series(dtype=int),'Map': pd.Series(dtype=object)})
    counter = 0
    for index, row in dataset.iterrows():
        line_of_strings = row['Poem']
        line_of_indexes = []
        for word in line_of_strings.split(" "):
            if word in states_map.keys():
                line_of_indexes.append(states_map[word])
        dataframe_indexed_set.loc[counter] = [index,line_of_indexes]
        counter +=1
    return dataframe_indexed_set
        


In [ ]:
poe_traning_indexed_df = transform_dataset_to_index_mapping(poe_training_data, states_in_poe)
poe_test_indexed_df = transform_dataset_to_index_mapping(poe_testing_data, states_in_poe)

frost_traning_indexed_df = transform_dataset_to_index_mapping(frost_training_data, states_in_frost)
frost_traning_indexed_df = transform_dataset_to_index_mapping(frost_training_data, states_in_frost)


In [ ]:
poe_traning_indexed_df.head()

,Line,Map
0,557,"[0, 1, 2, 3, 4, 5]"
1,505,"[6, 7, 8, 7, 9, 10, 11, 9, 12, 13, 14, 15, 16]"
2,578,"[17, 18, 19, 20, 11, 19, 21]"
3,659,"[11, 22, 16, 23, 24, 25]"
4,94,"[26, 27, 28, 13, 9, 29, 30, 31, 32, 33]"


Now we need to, for each dataset, train our markov model. In order to do that we have to create out matrix A and our vector pi. so
1st: we create a function that will generate a df correspondent to our matrix A with empty value. By now we only want the mould.
2nd: we create the transition matrix using the theory that we've learnt in the course. This will take some time
3rd: we also create a function for the initial state probability
4th: Now we need to group all these 3 processes and we will have a function that creates our markov model for a given dataset. 

With these done we're able to make predictions in a given sequence

In [ ]:
def create_skeleton_transitions_matrix(states):
    number_of_states = list(states.values())
    number_of_states.append('unknown')
    dictionary_dataframe = {}
    for state in number_of_states:
        dictionary_dataframe[state] = 1
    dictionary_dataframe['unknown'] = 1
    dataframe_transiciones_estados = pd.DataFrame(dictionary_dataframe, index=number_of_states)
    dataframe_initial_vector = pd.DataFrame.from_dict(dictionary_dataframe,orient='index')

    return dataframe_transiciones_estados, dataframe_initial_vector, len(number_of_states) 

In [ ]:
skeleton_A_poe = create_skeleton_transitions_matrix(states_in_poe)
skeleton_A_frost = create_skeleton_transitions_matrix(states_in_frost)
# print(skeleton_A_poe[1])

In [ ]:
def create_transition_matrix_and_initial_vector(sequences, matrix_skeleton, pi_vector):
    # Dataset debe ser la columna map del df que contiene los índices
    # Poe_training_indexed_df es el df que contiene las secuencias por indices [frase [1,2,3]]
    # Possible states es el diccionario 'palabra':indice ('but':0) poe_states.value    
    # dataframe_estados = create_skeleton_transitions_matrix(dataset)
    
    for sequence in sequences.iloc[:,1]:
        for index, element in enumerate(sequence):
            if index == 0:
                pi_vector.at[element,0] +=1
            else:
                previous_state = sequence[index-1]
                matrix_skeleton.at[previous_state,element] +=1
    # Now we have the vector as well as the initial state for the model with the add one smoothing but we need to make the division at each row
    transition_matrix = pd.DataFrame()
    for index, row in matrix_skeleton.iterrows():
        # In this case we don't need to add to the denominator anythong because the number of states is already there thanks to the add one smoothing
        denominator = np.sum(row,axis=0)
        new_row = row/denominator
        transition_matrix[index] = new_row

    return transition_matrix, pi_vector

In [ ]:
markov_poe = create_transition_matrix_and_initial_vector(poe_traning_indexed_df,skeleton_A_poe[0],skeleton_A_poe[1])


C:\Users\juan_\AppData\Local\Temp\ipykernel_13016\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_13016\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_13016\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

In [ ]:
markov_frost = create_transition_matrix_and_initial_vector(frost_traning_indexed_df, skeleton_A_frost[0],skeleton_A_frost[1])


C:\Users\juan_\AppData\Local\Temp\ipykernel_13016\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_13016\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transition_matrix[index] = new_row
C:\Users\juan_\AppData\Local\Temp\ipykernel_13016\1777347270.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

So at this point we have for each dataset a vector of initial states and a matrix with the values of all possible transitions. What we need now is a function that, receiving a sentence, tells us if the bot the probability of generate that sequence. The one with the higest probability should correspond to the model itself 

In [ ]:
markov_poe[1].loc['unknown'][0]

1

In [ ]:
def get_sentence_probability(sentence, model, index_mapping):
    splitted_sentence = sentence.split(" ")
    transformed_to_index_mapping = []
    for word in splitted_sentence:
        if word in index_mapping:
            transformed_to_index_mapping.append(index_mapping[word])
        else:
            transformed_to_index_mapping.append('unknown')
    # print("Esta es la secuencia ", transformed_to_index_mapping)
    for index, state in enumerate(transformed_to_index_mapping):
        probability = float()
        if index == 0:
            initial_probability = model[1].loc[state][index]
            probability = initial_probability
        else:
            transition_value = model[0].loc[transformed_to_index_mapping[index-1]][transformed_to_index_mapping[index]]
            probability = math.log(probability+1) * math.log(transition_value+1)
    return probability
            
        # Coger el valor del vector inicial para la primera palabra y los indices del df para las siguientes y después ejecutar la probabilidad


In [ ]:
def assign_author(sentence,model_1,model_2,index_mapping_1,index_mapping_2):
    probability_of_first_markov_model = get_sentence_probability(sentence, model_1, index_mapping_1)
    probability_of_second_markov_model = get_sentence_probability(sentence, model_2, index_mapping_2)
    if probability_of_first_markov_model > probability_of_second_markov_model:
        print("The sentence is from the first markov model (POE)")
    else:
        print("The sentence is from the second markov model (FROST)")

In [ ]:
test_poe_1 = poe_testing_data['Poem'][8]
test_poe_2 = poe_testing_data['Poem'][22]
test_poe_3 = poe_testing_data['Poem'][15]

test_frost_1 = frost_testing_data['Poem'][8]
test_frost_2 = frost_testing_data['Poem'][10]
test_frost_3 = frost_testing_data['Poem'][14]



In [ ]:
assign_author(test_poe_1,markov_poe,markov_frost,states_in_poe,states_in_frost)
assign_author(test_poe_2,markov_poe,markov_frost,states_in_poe,states_in_frost)
assign_author(test_poe_3,markov_poe,markov_frost,states_in_poe,states_in_frost)
assign_author(test_frost_1,markov_poe,markov_frost,states_in_poe,states_in_frost)
assign_author(test_frost_2,markov_poe,markov_frost,states_in_poe,states_in_frost)
assign_author(test_frost_3,markov_poe,markov_frost,states_in_poe,states_in_frost)

The sentence is from the second markov model (FROST)
The sentence is from the second markov model (FROST)
The sentence is from the second markov model (FROST)
The sentence is from the second markov model (FROST)
The sentence is from the second markov model (FROST)
The sentence is from the second markov model (FROST)


NO FUNCIONANDO. Intentar otro dia juntando todas las palabras en un mismo dataset (de ambos autores). El dataset deberá tener la linea y una etiqueta que identifique de quién es la línea. Con esto preparado dividir en test y train set y ejecutar los mismos pasos. (Es muy probable que siempre dé Poe porque en Frost hay más líneas y por lo tanto los estados están descompensados. Si solucionamos este problema no debería haber problema.)